In [1]:
from sympy.stats import DiscreteMarkovChain, TransitionMatrixOf
from sympy import Matrix, MatrixSymbol, Eq
from sympy.stats import P
from sympy.core.symbol import Str

In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('data/mRubis_dependencies.csv')
df

,Caller ID,Source (Caller),Target (Callee),Target ID
0,6,Authentication Service,Query Service,13
1,7,Availability Item Filter,Region Item Filter,15
2,1,Bid and Buy Service,Query Service,13
3,1,Bid and Buy Service,Persistence Service,17
4,1,Bid and Buy Service,Inventory Service,5
5,1,Bid and Buy Service,Authentication Service,6
6,8,Buy Now Item Filter,Availability Item Filter,7
7,9,Category Item Filter,Recommendation Item Filter,14
8,10,Comment Item Filter,Category Item Filter,9
9,5,Inventory Service,Query Service,13


In [4]:
combine_id_name = lambda id, name: (id, name)

callers = df['Caller ID'].combine(df['Source (Caller)'], combine_id_name)
targets = df['Target ID'].combine(df['Target (Callee)'], combine_id_name)

In [5]:
components = (callers.append(targets)).unique().tolist()
components.sort(key=lambda tup: tup[0])
components

[(1, 'Bid and Buy Service'),
 (2, 'Item Management Service'),
 (3, 'Reputation Service'),
 (4, 'User Management Service'),
 (5, 'Inventory Service'),
 (6, 'Authentication Service'),
 (7, 'Availability Item Filter'),
 (8, 'Buy Now Item Filter'),
 (9, 'Category Item Filter'),
 (10, 'Comment Item Filter'),
 (11, 'Last Second Sales Item Filter'),
 (12, 'Past Sales Item Filter'),
 (13, 'Query Service'),
 (14, 'Recommendation Item Filter'),
 (15, 'Region Item Filter'),
 (16, 'Seller Reputation Item Filter'),
 (17, 'Persistence Service'),
 (18, 'Future Sales Item Filter')]

In [6]:
# since floating point errors are a pain and need to be dealt with, just round every probability to 6 decimal places

def round_probability(prob):
    rounded = round(prob * 1000000)
    return rounded / 1000000

# also provide functionality to deal with any possible errors arising from the rounding

def validate_component(Mat, comps, component_start_idx):
    print('Component: {}'.format(comps[component_start_idx][1]))
    for j in range(3):
        row_idx = (3 * component_start_idx) + j
        print('Row: {:02d}, Sum: {}, Equal to 1? {}'.format(row_idx, sum(Mat.row(row_idx)), sum(Mat.row(row_idx)) == 1))
        if sum(Mat.row(row_idx)) != 1:
            correct_row(Mat, component_start_idx, j)
            print('Attempted to fix the row sum error. New Sum: {}, Equal to 1? {}'.format(sum(Mat.row(row_idx)), sum(Mat.row(row_idx)) == 1))

def validate_matrix(Mat, comps):
    for i in range(len(comps)):
        print('Component: {}'.format(comps[i][1]))
        for j in range(3):
            row_idx = (3 * i) + j
            print('Row: {:02d}, Sum: {}, Equal to 1? {}'.format(row_idx, sum(Mat.row(row_idx)), sum(Mat.row(row_idx)) == 1))
            if sum(Mat.row(row_idx)) != 1:
                correct_row(Mat, i, j)
                print('Fixed the row sum error. Row sum now equal to {}'.format(sum(Mat.row(row_idx))))


def correct_row(Mat, component_start_idx, state_idx):
        row_idx = (3 * component_start_idx) + state_idx

        if state_idx == 0:
            # reduce the self-edge of the op state
            Mat[(row_idx * 54) + (3 * component_start_idx)] = Mat[(row_idx * 54) + (3 * component_start_idx)] + (1 - sum(Mat.row(row_idx)))
        elif state_idx == 1:
            # reduce the self-edge of the dg state
            Mat[(row_idx * 54) + (3 * component_start_idx) + 1] = Mat[(row_idx * 54) + (3 * component_start_idx) + 1] + (1 - sum(Mat.row(row_idx)))
        elif state_idx == 2:
            # reduce the self-edge of the ur state
            Mat[(row_idx * 54) + (3 * component_start_idx) + 2] = Mat[(row_idx * 54) + (3 * component_start_idx) + 2] + (1 - sum(Mat.row(row_idx)))

In [7]:
# Markov Matrix

# first, create matrix of size 18*3 x 18*3 (as there are 3 states for each component)
# a row in this matrix corresponds to the outgoing edges/probabilities for one such internal state
# each row must sum up to 1 in the end
M = Matrix([
    [0.00 for i in range(len(components) * 3)] for j in range(len(components) * 3)
])

# now insert the internal probabilities, for transitions bewteen the internal states

# insert the probabilities for the outgoing edges from the operational states (first of three) to other internal states
for i in range(len(components)):
    # 3i * 54 leads us to the correct row, +3i to the correct column
    idx = (3 * i * 54) + (3 * i)
    M[idx] = round_probability(0.15)       # self-edge to operational state
    M[idx + 1] = round_probability(0.05)   # edge to the degraded state
    # remaining 80% are for external transitions and not assigned here

# insert probabilities for the outgoing edges from degraded states (second of three) to other internal states
for i in range(len(components)):
    # 3i * 54 + 54 leads to the correct row, + 3i to the column of the operational state of the same component
    idx = (3 * i * 54) + 54 + (3 * i)
    M[idx] = round_probability(0.5)        # to operational state
    M[idx + 1] = round_probability(0.25)   # self-edge to degraded state
    M[idx + 2] = round_probability(0.10)   # to unresponsive state
    # remaining 15% for external connections

# insert probabilities for the outgoing edges from unresponsive states (last of three) to other internal states
for i in range(len(components)):
    # 3i * 54 + 2 * 54 leads to the correct row, + 3i to the column of the operational state of the same component
    idx = (3 * i * 54) + (2 * 54) + (3 * i)
    M[idx] = round_probability(0.25)       # return to operational state
    M[idx + 2] = round_probability(0.5)    # stay in unresponsive state
    # remaining 25% for external connections

assert M.shape == (len(components)*3, len(components)*3), 'Markov Matrix has shape {}, but should have ({}, {})'.format(M.shape, len(components)*3, len(components)*3)

####################### possible break for debugging purposes ###############################

# next, calculate probabilities of a specific component transitioning to a different component
# use component IDs for later convenience

probs = Matrix([
    [0.00 for i in range(len(components))] for j in range(len(components))
])

for i in range(len(components)):
    # i+1 because the IDs start with 1, but i is starting at 0
    connections = df[df['Caller ID'] == i+1]
    for connection in connections.iterrows():
        # connection indices:
        #   0 -> connection index (useless)
        #   1 -> Data
        #        -> Caller ID
        #        -> Source (Caller)
        #        -> Target (Callee)
        #        -> Target ID
        # i*18 gets us into the correct row, Callee ID - 1 into the right column
        probs[(i * 18) + (connection[1]['Target ID'] - 1)] = 1.0 / len(connections)

####################### possible break for debugging purposes ###############################

# now use the probabilities of a component having an outgoing edge to another component to fill out Markov Matrix

# get dead end states - they get backward edges with 10% of the incoming probability, to not be sinks altogether
dead_end_states = []

for i in range(len(components)):
    if sum(probs.row(i)) == 0.0:
        dead_end_states.append(components[i][0])

####################### possible break for debugging purposes ###############################

# remember which nodes have connections to the dead end states for later use
connections_to_dead_end = dict()

# now do all outgoing edges for those components that are not dead ends first
for i in range(len(components)):
    if i + 1 in dead_end_states:
        continue

    op_row = 3 * i * 54
    dg_row = op_row + 54
    ur_row = op_row + (2 * 54)

    op_free = 1 - sum(M.row(3 * i))
    dg_free = 1 - sum(M.row((3 * i) + 1))
    ur_free = 1 - sum(M.row((3 * i) + 2))

    for j in range(len(components)):
        if j == i:
            # skip the self-edge, we already have that
            continue
        # set the operational state -> operational state connection
        M[op_row + (3 * j)] = round_probability(probs[(i * len(components)) + j] * op_free)

        # set the degraded -> degraded state connection
        M[dg_row + ((3 * j) + 1)] = round_probability(probs[(i * len(components)) + j] * dg_free)

        # set the unresponsive -> operational (half of free) and unresponsive -> unresponsive (half of free)  connections
        M[ur_row + (3 * j)] = round_probability(probs[(i * len(components)) + j] * (ur_free * 0.5))
        M[ur_row + ((3 * j) + 2)] = round_probability(probs[(i * len(components)) + j] * (ur_free * 0.5))

        # if connections to a dead end state were made, remember them for later creation of the backward edges
        # the unresponsive -> operational edge is not interesting as it is not
        if (j + 1) in dead_end_states:
            connections_to_dead_end[(i + 1, j + 1)] = [M[op_row + (3 * j)], M[dg_row + ((3 * j) + 1)], M[ur_row + (3 * j)], M[ur_row + ((3 * j) + 2)]]

    validate_component(M, components, i)

    assert sum(M.row(3 * i)) - 1.0 < 0.00000001, 'Sum of outgoing probabilities for the operational state of component {} was {}'.format(i+1, sum(M.row((3 * i))))
    assert sum(M.row((3 * i) + 1)) - 1.0 < 0.00000001, 'Sum of outgoing probabilities for the degraded state of component {} was {}'.format(i+1, sum(M.row((3 * i) + 1)))
    assert sum(M.row((3 * i) + 2)) - 1.0 < 0.00000001, 'Sum of outgoing probabilities for the unresponsive state of component {} was {}'.format(i+1, sum(M.row((3 * i) + 2)))

####################### possible break for debugging purposes ###############################

connections_to_dead_end = {k: v for (k, v) in connections_to_dead_end.items() if v != [0, 0, 0, 0]}

####################### possible break for debugging purposes ###############################

# create outgoing edges from the dead end states by adding backwards edges to their incoming ones with only a fraction
# of the probability. The remaining probabilities are distributed on the internal edges

# only the operational_de -> operational_src (regular operation) backwards edge is modeled. This decision is made due
# to the system not intending an outgoing connection from the dead end states in the beginning, thus propagating errors
# from them to any other state seems wrong

prob_fraction = 0.3

for i in dead_end_states:
    idx = i - 1     # index always one lower due to the IDs starting @ 1

    # the matrix row for the outgoing probabilities
    op_row = 3 * idx * 54

    # how large is the probability still required to be added to outgoing edges from the operational state?
    op_free = 1 - sum(M.row(3 * idx))

    # calculate the total of incoming probabilities for the operational state
    total = 0
    for (k, v) in connections_to_dead_end.items():
        if k[1] != i:
            continue
        total += v[0]

    # the total could be larger than 1.0 -> must ensure that the outgoing edges (prob_fraction * total) are within the
    # free probability from the operational state
    # if not, multiply all outgoing edges by 1 / ((prob_fraction * total) / op_free) to normalize to the maximum available probability
    norm_factor = (1 / ((prob_fraction * total) / op_free)) if (prob_fraction * total) > op_free else 1

    # add the backwards edges and remember their probability sum
    prob_sum = 0
    for (k, v) in connections_to_dead_end.items():
        if k[1] != i:
            continue

        backwards_prob = v[0] * prob_fraction * norm_factor

        # set the operational state -> operational state connection
        M[op_row + (3 * (k[0] - 1))] = backwards_prob

        prob_sum += backwards_prob

    # if the normalization factor is not 1, the remaining free probabilities for outgoing edges from the operational
    # state are fully depleted, however, if it is one there may be some more probabilities left that need distributing
    if norm_factor == 1:
        remainder = op_free - prob_sum
        # get total of internal outgoing edges to distribute the remaining probabilities
        op_self_edge_idx = (3 * idx * 54) + (3 * idx)
        sum_internal = M[op_self_edge_idx] + M[op_self_edge_idx + 1] + M[op_self_edge_idx + 2]

        # distribute
        M[op_self_edge_idx] += round_probability((M[op_self_edge_idx] / sum_internal) * remainder)
        M[op_self_edge_idx + 1] += round_probability((M[op_self_edge_idx + 1] / sum_internal) * remainder)
        M[op_self_edge_idx + 2] += round_probability((M[op_self_edge_idx + 2] / sum_internal) * remainder)

    # now also distribute the remaining probabilities of the degraded and unresponsive states to their internal connections
    dg_remainder = 1 - sum(M.row((3 * idx) + 1))
    ur_remainder = 1 - sum(M.row((3 * idx) + 2))

    dg_op_edge_idx = (3 * idx * 54) + 54 + (3 * idx)
    ur_op_edge_idx = (3 * idx * 54) + (2 * 54) + (3 * idx)

    # totals
    sum_internal_dg = M[dg_op_edge_idx] + M[dg_op_edge_idx + 1] + M[dg_op_edge_idx + 2]
    sum_internal_ur = M[ur_op_edge_idx] + M[ur_op_edge_idx + 1] + M[ur_op_edge_idx + 2]

    # distribute degraded
    M[dg_op_edge_idx] += round_probability((M[dg_op_edge_idx] / sum_internal_dg) * dg_remainder)
    M[dg_op_edge_idx + 1] += round_probability((M[dg_op_edge_idx + 1] / sum_internal_dg) * dg_remainder)
    M[dg_op_edge_idx + 2] += round_probability((M[dg_op_edge_idx + 2] / sum_internal_dg) * dg_remainder)

    # distribute unresponsive
    M[ur_op_edge_idx] += (M[ur_op_edge_idx] / sum_internal_ur) * ur_remainder
    M[ur_op_edge_idx + 1] += (M[ur_op_edge_idx + 1] / sum_internal_ur) * ur_remainder
    M[ur_op_edge_idx + 2] += (M[ur_op_edge_idx + 2] / sum_internal_ur) * ur_remainder

    validate_component(M, components, idx)

    assert sum(M.row(3 * idx)) - 1.0 < 0.00000001, 'Sum of outgoing probabilities for the operational state of component {} was {}'.format(i, sum(M.row((3 * idx))))
    assert sum(M.row((3 * idx) + 1)) - 1.0 < 0.00000001, 'Sum of outgoing probabilities for the degraded state of component {} was {}'.format(i, sum(M.row((3 * idx) + 1)))
    assert sum(M.row((3 * idx) + 2)) - 1.0 < 0.00000001, 'Sum of outgoing probabilities for the unresponsive state of component {} was {}'.format(i, sum(M.row((3 * idx) + 2)))

####################### possible break for debugging purposes ###############################

# deal with any possible errors left in the matrix row sums (shouldn't be any, but just to be sure)
validate_matrix(M, components)


Component: Bid and Buy Service
Row: 00, Sum: 1.00000000000000, Equal to 1? True
Row: 01, Sum: 1.00000000000000, Equal to 1? False
Attempted to fix the row sum error. New Sum: 1.00000000000000, Equal to 1? True
Row: 02, Sum: 1.00000000000000, Equal to 1? True
Component: Item Management Service
Row: 03, Sum: 1.00000100000000, Equal to 1? False
Attempted to fix the row sum error. New Sum: 1.00000000000000, Equal to 1? True
Row: 04, Sum: 1.00000000000000, Equal to 1? True
Row: 05, Sum: 1.00000200000000, Equal to 1? False
Attempted to fix the row sum error. New Sum: 1.00000000000000, Equal to 1? True
Component: Reputation Service
Row: 06, Sum: 1.00000100000000, Equal to 1? False
Attempted to fix the row sum error. New Sum: 1.00000000000000, Equal to 1? True
Row: 07, Sum: 1.00000000000000, Equal to 1? True
Row: 08, Sum: 1.00000200000000, Equal to 1? False
Attempted to fix the row sum error. New Sum: 1.00000000000000, Equal to 1? True
Component: User Management Service
Row: 09, Sum: 1.0000010

In [8]:
# finally, we have the finished markov matrix
# now, define the DTMC
state_space = []
for component in components:
    state_space.extend([component[1] + ' op', component[1] + ' dg', component[1] + ' ur'])

state_space = [Str(state) for state in state_space]


Y = DiscreteMarkovChain('Y', state_space, M)

In [9]:
P(Eq(Y[1], Str("Availability Item Filter op")), Eq(Y[0], Str("Buy Now Item Filter op")))


4/5

In [10]:
P(Eq(Y[1], Str("Future Sales Item Filter dg")), Eq(Y[0], Str("Future Sales Item Filter op")))

0.190000000000000

In [35]:
def generate_stack_trace(state_space, transition_matrix, trace_length, start_state):
    stack = []
    transit_probs = [1.0]

    nrows = transition_matrix.shape[0]
    ncols = transition_matrix.shape[1]

    prob_mat = dict()

    for i in range(nrows):
        probs = []
        for j in range(ncols):
            probs.append(transition_matrix[(i * ncols) + j])
        prob_mat[state_space[i]] = probs

    curr_state = start_state

    while len(stack) < trace_length:
        stack.append(curr_state)
        next_state = np.random.choice(state_space, 1, p=prob_mat[curr_state])
        transit_probs.append(prob_mat[curr_state][state_space.index(next_state[0])])
        curr_state = next_state[0]

    return stack, transit_probs

In [36]:
trace, trace_probs = generate_stack_trace(state_space, M, 30, Str('Bid and Buy Service op'))

In [42]:
for i in range(len(trace)):
    print('With {} prob to state {}'.format(trace_probs[i], trace[i]))

With 1.0 prob to state Bid and Buy Service op
With 1/5 prob to state Query Service op
With 4/5 prob to state Last Second Sales Item Filter op
With 4/5 prob to state Past Sales Item Filter op
With 4/5 prob to state Buy Now Item Filter op
With 4/5 prob to state Availability Item Filter op
With 4/5 prob to state Region Item Filter op
With 4/5 prob to state Seller Reputation Item Filter op
With 4/5 prob to state Comment Item Filter op
With 4/5 prob to state Category Item Filter op
With 0.150000000000000 prob to state Category Item Filter op
With 0.150000000000000 prob to state Category Item Filter op
With 4/5 prob to state Recommendation Item Filter op
With 4/5 prob to state Future Sales Item Filter op
With 0.190000000000000 prob to state Future Sales Item Filter dg
With 0.588235000000000 prob to state Future Sales Item Filter op
With 0.240000000000000 prob to state Recommendation Item Filter op
With 4/5 prob to state Future Sales Item Filter op
With 0.240000000000000 prob to state Recomme